In [5]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [ ]:
from ExMAS.utils import inData as inData

In [30]:
to_apply = pd.DataFrame(np.random.randint(0,10000,size=(10000, 4)), columns=list('ABCD'))
to_apply['degree'] = to_apply.apply(lambda x: 1 if x.name % 2 == 0 else 2, axis = 1)
to_apply

,A,B,C,D,degree
0,2797,2649,5536,3007,1
1,5867,7586,177,2442,2
2,1454,6347,5238,5720,1
3,6176,4314,8364,2529,2
4,3663,4094,957,6666,1
...,...,...,...,...,...
9995,7467,4826,1784,701,2
9996,8571,3529,8880,3217,1
9997,2798,4219,1637,6966,2
9998,1839,358,6587,2367,1


In [31]:
ddf = dd.from_pandas(to_apply[to_apply.degree != 1], npartitions=4)
ddf

,A,B,C,D,degree
npartitions=4,,,,,
1,int64,int64,int64,int64,int64
2501,...,...,...,...,...
5001,...,...,...,...,...
7501,...,...,...,...,...
9999,...,...,...,...,...


In [32]:
def transitize(inData, ride):
    ret = pd.Series({'indexes': None,
                     'origin': None,
                     'destination': None,
                     'treq': None,
                     'df': None,
                     'transitizable': False,
                     'ttrav': None,
                     'u_veh': None,
                     'times': None,
                     'dist': None,
                     'orig_walk_time': None,
                     'dest_walk_time': None,
                     'u_pax': None,
                     'u_paxes': None,
                     'efficient': False,
                     'VoT': False,
                     })
    if ride['degree'] == 1:
        return ret  # not applicable for single rides
    else:
        ret.dist = inData.passengers.shape[0]
        ret.ttrav = np.random.randint(0,100)
        ret.origin = pd.util.testing.rands_array(10, 1)[0]
        ret.u_paxes = [0,3,4,5]
        ret.VoT = 15
        return ret
        

In [39]:
ddf = dd.from_pandas(to_apply[to_apply.degree != 1], npartitions=4)
applied = ddf.map_partitions(lambda dframe: dframe.apply(lambda x: transitize(inData, x), axis=1)).compute(
            scheduler='processes')

In [41]:
applied.dtypes

indexes           object
origin            object
destination       object
treq              object
df                object
transitizable       bool
ttrav              int64
u_veh             object
times             object
dist               int64
orig_walk_time    object
dest_walk_time    object
u_pax             object
u_paxes           object
efficient           bool
VoT                int64
dtype: object

In [33]:
ddf = ddf.map_partitions(lambda dframe: dframe.apply(lambda x: transitize(inData, x), axis=1))

In [35]:
ret = ddf.compute(scheduler='processes')

In [36]:
ret

,indexes,origin,destination,treq,df,transitizable,ttrav,u_veh,times,dist,orig_walk_time,dest_walk_time,u_pax,u_paxes,efficient,VoT
1,None,1xHHZlDrbL,None,None,None,False,8,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
3,None,kPoTXKXbTz,None,None,None,False,11,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
5,None,EGFRVlOnGN,None,None,None,False,85,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
7,None,h8EOT1hoKw,None,None,None,False,12,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
9,None,vaKwxEznJU,None,None,None,False,60,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,None,SDMUpgjCSe,None,None,None,False,96,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
9993,None,MbX3mWYWid,None,None,None,False,52,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
9995,None,qLTQOvuxgj,None,None,None,False,62,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
9997,None,jxxLg8F4m0,None,None,None,False,56,None,None,0,None,None,None,"[0, 3, 4, 5]",False,15
